## source code

In [1]:
# The following code is copied from https://github.com/zyt717/CTFCodecTool/blob/master/ctfcodecs/tudoucode.py
from Crypto.Cipher import AES
from random import choice, randint

KEY = b'XDXDtudou@KeyFansClub^_^Encode!!'
IV = b'Potato@Key@_@=_='

TUDOU = [
    '滅', '苦', '婆', '娑', '耶', '陀', '跋', '多', '漫', '都', '殿', '悉', '夜', '爍', '帝', '吉',
    '利', '阿', '無', '南', '那', '怛', '喝', '羯', '勝', '摩', '伽', '謹', '波', '者', '穆', '僧',
    '室', '藝', '尼', '瑟', '地', '彌', '菩', '提', '蘇', '醯', '盧', '呼', '舍', '佛', '參', '沙',
    '伊', '隸', '麼', '遮', '闍', '度', '蒙', '孕', '薩', '夷', '迦', '他', '姪', '豆', '特', '逝',
    '朋', '輸', '楞', '栗', '寫', '數', '曳', '諦', '羅', '曰', '咒', '即', '密', '若', '般', '故',
    '不', '實', '真', '訶', '切', '一', '除', '能', '等', '是', '上', '明', '大', '神', '知', '三',
    '藐', '耨', '得', '依', '諸', '世', '槃', '涅', '竟', '究', '想', '夢', '倒', '顛', '離', '遠',
    '怖', '恐', '有', '礙', '心', '所', '以', '亦', '智', '道', '。', '集', '盡', '死', '老', '至']

BYTEMARK = ['冥', '奢', '梵', '呐', '俱', '哆', '怯', '諳', '罰', '侄', '缽', '皤']


def Encrypt(plaintext):
    # 1. Encode Plaintext in UTF-16 Little Endian
    data = plaintext.encode('utf-16le')
    # 2. Add Paddings (PKCS7)
    pads = (- len(data)) % 16
    data = data + bytes(pads * [pads])
    # 3. Use AES-256-CBC to Encrypt
    cryptor = AES.new(KEY, AES.MODE_CBC, IV)
    result = cryptor.encrypt(data)
    # 4. Encode and Add Header
    return '佛曰：' + ''.join([TUDOU[i] if i < 128 else choice(BYTEMARK) + TUDOU[i - 128] for i in result])


def Decrypt(ciphertext):
    # 1. Remove Header and Decode
    if ciphertext.startswith('佛曰：'):
        ciphertext = ciphertext[3:]
        data = b''
        i = 0
        while i < len(ciphertext):
            if ciphertext[i] in BYTEMARK:
                i = i + 1
                data = data + bytes([TUDOU.index(ciphertext[i]) + 128])
            else:
                data = data + bytes([TUDOU.index(ciphertext[i])])
            i = i + 1
        # 2. Use AES-256-CBC to Decrypt
        cryptor = AES.new(KEY, AES.MODE_CBC, IV)
        result = cryptor.decrypt(data)
        # 3. Remove Paddings (PKCS7)
        flag = result[-1]
        if flag < 16 and result[-flag] == flag:
            result = result[:-flag]
        # 4. Decode Plaintext with UTF-16 Little Endian
        return result.decode('utf-16le')
    else:
        return ''


In [ ]:
if __name__ == '__main__':
    with open("flag1","r",encoding="utf-8") as f:
        x=f.read()
    with open("flag1.enc","w",encoding="utf-8") as f:
        f.write(Encrypt(x).encode("utf-8").decode("shift_jis",errors="ignore"))
    with open("flag2","rb") as f:
        x=f.read()
    from base64 import b16encode, b32encode, b64encode, b85encode, a85encode
    for i in range(10):
        x=choice([b16encode, b32encode, b64encode, b85encode, a85encode])(x)
    with open("flag2.enc","w",encoding="utf-8") as f:
        f.write(Encrypt(x.decode("utf-8")).encode("utf-8").decode("shift_jis",errors="ignore"))

## flag2

In [2]:
with open("flag2.enc","r",encoding="utf-8") as f:
    s = f.readline()

In [3]:
utf_full = s.encode("shift_jis")
utf = s.encode("shift_jis")[8:]

In [4]:
xe = []
for idx,x in enumerate(utf):
    if x >= 0xe0:
        xe.append(idx)
total = len(xe)

In [5]:
total

27466

In [6]:
from collections import Counter
Counter([xe[i+1]-xe[i] for i in range(len(xe)-1)])

Counter({2: 4712, 3: 22670, 1: 83})

In [7]:
dic = TUDOU + BYTEMARK
dic = [x.encode("utf-8") for x in dic]

ins = set()
for x in dic:
    for xx in x:
        ins.add(xx)
        
ins = [x.to_bytes(1,"little") for x in ins]

In [8]:
from tqdm import tqdm
possible = [[] for i in range(len(xe))]
for i in tqdm(range(total)):
    if i < total - 1:
        ss = utf[xe[i]:xe[i+1]]
    else:
        ss = utf[xe[i]:]
    
    if len(ss) == 3:
        possible[i].append(ss.decode("utf-8"))
    
    elif len(ss) == 2:
        for x in ins:
            byte = ss[:1]+x+ss[1:2]
            if byte in dic:
                possible[i].append(byte.decode("utf-8"))

            byte = ss[:2]+x
            if byte in dic:
                possible[i].append(byte.decode("utf-8"))
    
    elif len(ss) == 1:
        for x in ins:
            for y in ins:
                byte = ss[:1]+x+y
                if byte in dic:
                    possible[i].append(byte.decode("utf-8"))

100%|██████████| 27466/27466 [00:01<00:00, 14070.95it/s]


In [9]:
Counter([len(x) for x in possible])

Counter({1: 24272, 2: 2329, 4: 107, 3: 549, 5: 126, 25: 51, 36: 32})

In [35]:
def test_possible(cur, idx, step):
    if step == 0 or idx == total:
        if utf_full.startswith(cur.encode("utf-8").decode("shift_jis",errors="ignore").encode("shift_jis")):
            return True
        else:
            return False
    
    for x in possible[idx]:
        if test_possible(cur+x, idx+1, step-1):
            return True
    return False

real_possible = [[] for i in range(len(xe))]
example = "佛曰："
for idx,p in tqdm(enumerate(possible)):
    for x in p:
        if test_possible(example+x, idx+1, 5):
            real_possible[idx].append(x)
    example += real_possible[idx][0]

27466it [00:29, 942.41it/s] 


In [36]:
Counter([len(x) for x in real_possible])

Counter({1: 27409, 2: 55, 3: 2})

In [37]:
example.encode("utf-8").decode("shift_jis",errors="ignore").encode("shift_jis") == utf_full

True

In [38]:
len(example)

27469

In [39]:
sst = [idx for idx,p in enumerate(real_possible) if len(p)>=2]

In [40]:
def half_decrypt(ciphertext):
    # 1. Remove Header and Decode
    if ciphertext.startswith('佛曰：'):
        ciphertext = ciphertext[3:]
        data = b''
        i = 0
        while i < len(ciphertext):
            if ciphertext[i] in BYTEMARK:
                i = i + 1
                data = data + bytes([TUDOU.index(ciphertext[i]) + 128])
            else:
                data = data + bytes([TUDOU.index(ciphertext[i])])
            i = i + 1
        # 2. Use AES-256-CBC to Decrypt
        cryptor = AES.new(KEY, AES.MODE_CBC, IV)
        result = cryptor.decrypt(data)
        return result

In [20]:
# 根据末尾数值做对应
cipher = example
dec = []

for _ in tqdm(range(1000)):
    i = choice(sst)
    idx = i + 3
    for x in real_possible[i]:
        if x != cipher[idx]:
            cipher = cipher[:idx] + x + cipher[idx+1:]
            if cipher.encode("utf-8").decode("shift_jis",errors="ignore").encode("shift_jis") != utf_full:
                print("error!")
                break
            try:
                res = Decrypt(cipher)
                dec.append(res)
                print(len(dec))
            except:
                pass

100%|██████████| 1000/1000 [01:13<00:00, 13.57it/s]


In [41]:
len(example), len(possible)

(27469, 27466)

In [42]:
def change_one(cipher, i, pos="first"):
    idx = i + 3
    for x in real_possible[i]:
        if x != cipher[idx]:
            if len(real_possible[i]) == 3 and pos=="last":
                pos = "first"
                continue
            cipher = cipher[:idx] + x + cipher[idx+1:]
            if cipher.encode("utf-8").decode("shift_jis",errors="ignore").encode("shift_jis") != utf_full:
                print("error!")
                break
            return cipher
            

In [82]:
mem = cipher

In [164]:
def first_err(s):
    for idx,x in enumerate(s):
        if ord(x)>128:
            return idx
    return 1e8

In [165]:
cipher = mem
for idx,i in enumerate(sst):
    ss = Decrypt(cipher)
    p = first_err(ss)
    tmp = change_one(cipher, i)
    try:
        ss = Decrypt(tmp)
        if first_err(ss) > p:
            cipher = tmp
            p = first_err(ss)
            print(i,p)
    except:
        pass
        
    try:
        tmp = change_one(cipher, i)
        ss = Decrypt(tmp)
        if first_err(ss) > p:
            cipher = tmp
            p = first_err(ss)
            print(i,p)
    except:
        pass

378 832
2515 1840
5524 2024
6052 2232
6684 2784
8317 3384
10110 4176
12476 4496
13451 5056
15125 5344
16025 5784
17318 6304
18879 6440
19257 6752
20204 6760
20233 7408
22193 7560
22638 7840
23480 8096
24221 8168
24448 8192
24517 8464
25327 8656
25914 9160
27430 100000000.0


In [214]:
ss = Decrypt(cipher).encode("utf-8")

In [215]:
from base64 import b16decode, b32decode, b64decode, b85decode, a85decode

In [216]:
for i in range(10):
    for dec in [b16decode, b32decode, b64decode, b85decode, a85decode]:
        try:
            tmp = dec(ss)
            flag = True
            for x in tmp:
                if x > 128:
                    flag = False
                    break
            if flag:
                ss = tmp
                print(i, dec)
                break
        except:
            continue

0 <function b32decode at 0x7f2c02727d30>
1 <function a85decode at 0x7f2c0262c040>
2 <function b85decode at 0x7f2c0262c160>
3 <function b16decode at 0x7f2c02727e50>
4 <function a85decode at 0x7f2c0262c040>
5 <function b64decode at 0x7f2c027279d0>
6 <function a85decode at 0x7f2c0262c040>
7 <function b64decode at 0x7f2c027279d0>
8 <function a85decode at 0x7f2c0262c040>


In [222]:
b64decode(ss).decode("utf-8")

'将需要打码的文字输入在上面的文本框里，点击『听佛说宇宙的真谛』按钮，就能在下面得到打码后的文字。\r\nflag{AES_1s_b10ck_cipher}\r\n将需要解码的文字输入在下面的文本框里，记得带上『佛曰：』或『如是我闻：』的文字，点击『参悟佛所言的真意』按钮，就能在上面的文本框里得到解码后的文字。'

## flag1

In [16]:
for x in sorted(dic):
    print(x, x.decode("utf-8"))

b'\xe3\x80\x82' 。
b'\xe4\xb8\x80' 一
b'\xe4\xb8\x89' 三
b'\xe4\xb8\x8a' 上
b'\xe4\xb8\x8d' 不
b'\xe4\xb8\x96' 世
b'\xe4\xba\xa6' 亦
b'\xe4\xbb\x96' 他
b'\xe4\xbb\xa5' 以
b'\xe4\xbc\x8a' 伊
b'\xe4\xbc\xbd' 伽
b'\xe4\xbd\x9b' 佛
b'\xe4\xbe\x84' 侄
b'\xe4\xbe\x9d' 依
b'\xe4\xbf\xb1' 俱
b'\xe5\x80\x92' 倒
b'\xe5\x83\xa7' 僧
b'\xe5\x86\xa5' 冥
b'\xe5\x88\x87' 切
b'\xe5\x88\xa9' 利
b'\xe5\x8b\x9d' 勝
b'\xe5\x8d\x97' 南
b'\xe5\x8d\xb3' 即
b'\xe5\x8f\x83' 參
b'\xe5\x90\x89' 吉
b'\xe5\x91\x90' 呐
b'\xe5\x91\xbc' 呼
b'\xe5\x92\x92' 咒
b'\xe5\x93\x86' 哆
b'\xe5\x96\x9d' 喝
b'\xe5\x9c\xb0' 地
b'\xe5\xa4\x9a' 多
b'\xe5\xa4\x9c' 夜
b'\xe5\xa4\xa2' 夢
b'\xe5\xa4\xa7' 大
b'\xe5\xa4\xb7' 夷
b'\xe5\xa5\xa2' 奢
b'\xe5\xa7\xaa' 姪
b'\xe5\xa8\x91' 娑
b'\xe5\xa9\x86' 婆
b'\xe5\xad\x95' 孕
b'\xe5\xae\xa4' 室
b'\xe5\xaf\x86' 密
b'\xe5\xaf\xa6' 實
b'\xe5\xaf\xab' 寫
b'\xe5\xb0\xbc' 尼
b'\xe5\xb8\x9d' 帝
b'\xe5\xba\xa6' 度
b'\xe5\xbd\x8c' 彌
b'\xe5\xbe\x97' 得
b'\xe5\xbf\x83' 心
b'\xe6\x80\x96' 怖
b'\xe6\x80\x9b' 怛
b'\xe6\x80\xaf' 怯
b'\xe6\x81\x90' 恐
b'\xe6\x82

In [20]:
with open("flag1.enc","r",encoding="utf-8") as f:
    s = f.readline()

In [21]:
utf = s.encode("shift_jis")[8:]

In [22]:
dic = TUDOU + BYTEMARK + ["："]
dic = [x.encode("utf-8") for x in dic]

ins = set()
for x in dic:
    for xx in x:
        ins.add(xx)
        
ins = [x.to_bytes(1,"little") for x in ins]  # 0x80-0xbf, 0xe3-0xe9

In [24]:
def recur(output, ss):
    #print(len(ss), output, ss[:10])
    if not utf.startswith(output.encode("utf-8").decode("shift_jis",errors="ignore").encode("shift_jis")):
        return
    
    if len(ss) == 0:
        possible.append(output)
    
    byte = ss[0:3]
    if byte in dic:
        recur(output+byte.decode("utf-8"), ss[3:])
        
    for x in ins:
        byte = x+ss[:2]
        if byte in dic:
            recur(output+byte.decode("utf-8"), ss[2:])
            
        byte = ss[:1]+x+ss[1:2]
        if byte in dic:
            recur(output+byte.decode("utf-8"), ss[2:])
            
        byte = ss[:2]+x
        if byte in dic:
            recur(output+byte.decode("utf-8"), ss[2:])
            
    for x in ins:
        for y in ins:
            byte = x+y+ss[:1]
            if byte in dic:
                recur(output+byte.decode("utf-8"), ss[1:])
                
            byte = x+ss[:1]+y
            if byte in dic:
                recur(output+byte.decode("utf-8"), ss[1:])
                
            byte = ss[:1]+x+y
            if byte in dic:
                recur(output+byte.decode("utf-8"), ss[1:])

In [25]:
possible = []
recur("", utf)

In [26]:
possible

['麼是梵殿皤盡皤滅娑苦冥那盧羯伊罰智盡夜梵勝俱實利盧皤呼呐。帝夢梵滅羅實呐等依皤蘇侄倒伊以亦哆數怯姪尼怯無呐明倒神怯夷遠侄摩智呐伽恐怯亦冥彌缽一',
 '麼是梵殿皤盡皤滅娑苦冥那盧羯伊罰智盡夜梵勝俱實利盧皤呼呐。帝夢梵滅羅實呐等依皤蘇侄倒伊以亦哆數怯姪尼怯無呐明倒神怯夷遠侄摩智呐伽恐怯亦冥彌缽三',
 '麼是梵殿皤盡皤滅娑苦冥那盧羯伊罰智盡夜梵勝俱實利盧皤呼呐。帝夢梵滅羅實呐等依皤蘇侄倒伊以亦哆數怯姪尼怯無呐明倒神怯夷遠侄摩智呐伽恐怯亦冥彌缽上',
 '麼是梵殿皤盡皤滅娑苦冥那盧羯伊罰智盡夜梵勝俱實利盧皤呼呐。帝夢梵滅羅實呐等依皤蘇侄倒伊以亦哆數怯姪尼怯無呐明倒神怯夷遠侄摩智呐伽恐怯亦冥彌缽不',
 '麼是梵殿皤盡皤滅娑苦冥那盧羯伊罰智盡夜梵勝俱實利盧皤呼呐。帝夢梵滅羅實呐等依皤蘇侄倒伊以亦哆數怯姪尼怯無呐明倒神怯夷遠侄摩智呐伽恐怯亦冥彌缽世']

In [27]:
rev_TUDOU = {x:i for i,x in enumerate(TUDOU)}

for p in possible:
    hant = zhconv.convert(p, "zh-hant") # 这里不需要转化为繁体

    origin = b""
    flag = 0
    for x in hant:
        if x in rev_TUDOU:
            origin += (rev_TUDOU[x]+flag*128).to_bytes(1,"little")
            flag = 0
        else:
            if x in BYTEMARK:
                flag = 1
            else:
                flag = 1 
                print("error", x)
    
    cryptor = AES.new(KEY, AES.MODE_CBC, IV)
    result = cryptor.decrypt(origin)
    
    
    flag = result[-1]
    if flag < 16 and result[-flag] == flag:
        result = result[:-flag]
        
    try:
        print(result.decode('utf-16le'))
    except:
        pass


error 吶
error 吶
error 吶
error 吶
flag{s1mp1e_Tud0銲ꅙ揲锔睟챽ᄻ
error 吶
error 吶
error 吶
error 吶
flag{s1mp1e_Tud0룄爉鸆몠㊆硎ữ
error 吶
error 吶
error 吶
error 吶
error 吶
error 吶
error 吶
error 吶
flag{s1mp1e_Tud0uc0d3}
error 吶
error 吶
error 吶
error 吶
flag{s1mp1e_Tud0⧶䳐鑅绎箄䰍໎
